In [67]:
import pandas as pd
import re
import nltk
import pymorphy2
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from gensim import corpora, models
from bertopic import BERTopic

In [68]:
nltk.download('punkt')
nltk.download('stopwords')

custom_stopwords = {
    'это', 'очень', 'просто', 'ещё', 'такой', 'нужно', 'вообще', 'чтото', 'что',
    'которые', 'было', 'будет', 'можно', 'нельзя', 'только', 'самый',
    'нам', 'вам', 'без', 'при', 'всё', 'него', 'она', 'они', 'какой', 'разные', 'другие','некоторые', 'быть'
}
stop_words = set(stopwords.words('russian')) | custom_stopwords

synonyms = {
    "задача": "задание",
    "задачи": "задание",
    "задач": "задание",
    "заданий": "задание"
}

morph = pymorphy2.MorphAnalyzer()

def preprocess(text):
    text = text.lower()
    text = re.sub(r'[^а-яa-z\s]', '', text)
    tokens = word_tokenize(text)
    lemmas = []
    for token in tokens:
        if token not in stop_words and len(token) > 2:
            lemma = morph.parse(token)[0].normal_form
            lemma = synonyms.get(lemma, lemma)
            if lemma not in stop_words:
                lemmas.append(lemma)
    return lemmas

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [72]:
pos_df = pd.read_excel("pos_reviews.xlsx")
neg_df = pd.read_excel("neg_reviews.xlsx")

pos_texts = pos_df['review'].dropna().tolist()
neg_texts = neg_df['review'].dropna().tolist()

pos_tokenized = [preprocess(text) for text in pos_texts]
neg_tokenized = [preprocess(text) for text in neg_texts]

In [110]:
def lda_analysis(tokenized_texts, title):
    dictionary = corpora.Dictionary(tokenized_texts)
    corpus = [dictionary.doc2bow(text) for text in tokenized_texts]
    lda_model = models.LdaModel(corpus=corpus, id2word=dictionary, num_topics=5, passes=15)

    print(f"\nТемы по LDA: {title}")
    for idx, topic in lda_model.print_topics(-1):
        print(f"Тема {idx+1}: {topic}")

lda_analysis(pos_tokenized, "Позитивные")
lda_analysis(neg_tokenized, "Негативные")


Темы по LDA: Позитивные
Тема 1: 0.073*"курс" + 0.017*"задание" + 0.016*"хороший" + 0.014*"отличный" + 0.011*"знание" + 0.009*"материал" + 0.009*"автор" + 0.009*"сложный" + 0.009*"понравиться" + 0.008*"практика"
Тема 2: 0.057*"курс" + 0.019*"задание" + 0.017*"весь" + 0.017*"отличный" + 0.016*"интересный" + 0.013*"работа" + 0.011*"хороший" + 0.011*"материал" + 0.010*"полезный" + 0.009*"рекомендовать"
Тема 3: 0.066*"курс" + 0.037*"задание" + 0.013*"материал" + 0.013*"сложный" + 0.012*"отличный" + 0.011*"подача" + 0.011*"хороший" + 0.010*"спасибо" + 0.010*"практический" + 0.009*"практика"
Тема 4: 0.027*"курс" + 0.016*"работа" + 0.012*"дать" + 0.008*"системный" + 0.008*"возможно" + 0.004*"аналитик" + 0.004*"далёкий" + 0.004*"приглашение" + 0.004*"классный" + 0.004*"немного"
Тема 5: 0.031*"курс" + 0.018*"понравиться" + 0.016*"тема" + 0.013*"задание" + 0.011*"материал" + 0.008*"урок" + 0.008*"интересный" + 0.008*"возможно" + 0.006*"легко" + 0.006*"подробно"

Темы по LDA: Негативные
Тема 1: 0

In [109]:
def bertopic_analysis(texts, title):
    tokenized = [preprocess(text) for text in texts]
    cleaned_texts = [" ".join(tokens) for tokens in tokenized]

    model = BERTopic(language="multilingual", min_topic_size=2)
    topics, _ = model.fit_transform(cleaned_texts)

    print(f"\nТопики BERTopic ({title}):")
    print(model.get_topic_info().head())

    return model, topics, cleaned_texts

topic_model_pos, topics_pos, pos_cleaned = bertopic_analysis(pos_texts, "Позитивные")
topic_model_neg, topics_neg, neg_cleaned = bertopic_analysis(neg_texts, "Негативные")


Топики BERTopic (Позитивные):
   Topic  Count                                        Name  \
0     -1     27              -1_сложный_задание_знание_курс   
1      0     13  0_профессия_тестировщик_возможность_работа   
2      1     10         1_автор_приятный_курс_благодарность   
3      2      9        2_видео_хороший_особенно_видеоразбор   
4      3      8            3_комментарий_скучно_брать_целое   

                                      Representation  \
0  [сложный, задание, знание, курс, подробно, раз...   
1  [профессия, тестировщик, возможность, работа, ...   
2  [автор, приятный, курс, благодарность, большой...   
3  [видео, хороший, особенно, видеоразбор, воспри...   
4  [комментарий, скучно, брать, целое, момент, ре...   

                                 Representative_Docs  
0  [ооп курс узнать узнавать неинтересно скучно м...  
1  [отличный курс профориентация данный курс помо...  
2  [отличный курс приятный чистый речь автор дост...  
3  [хороший курс сначала сложно в